In [16]:
import wandb

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM,Conv1D,MaxPool1D,Bidirectional
from tensorflow.keras.models import Model

from TranscriptionDataset.TensorflowLoader import MusicMetaSequence

from TUtils import get_timestamp

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


In [ ]:
wandb.init(project='TFArrangement')

In [3]:
trainSequence = MusicMetaSequence("Trainsets/S_Tier_Mel.hdf5")
testSequence = MusicMetaSequence("Trainsets/Mini_Validation_Test.hdf5",shuffle=False)
# trainSequence[0][0].shape,trainSequence[0][1].shape

In [4]:
input_layer = Input(shape=(None,256),batch_size=1)
x = (Conv1D(data_format="channels_last",filters=128, kernel_size=5, padding="same", activation="relu"))(input_layer)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=64, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=8, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = Bidirectional(LSTM(128))(x)
x = Dense(128,activation="relu")(x)
# output = Dense(1,activation="sigmoid")(x)
outputs = []
for i in range(6):
    outputs.append(Dense(1,name=f'output_{i}',activation="sigmoid")(x))

model = Model(inputs=input_layer, outputs=outputs)

print(model.summary())

2023-06-20 20:19:10.439422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:19:10.443775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:19:10.443838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:19:10.446057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:19:10.446112: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, None, 256)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (1, None, 128)       163968      ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (1, None, 128)       0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (1, None, 64)        41024       ['max_pooling1d[0][0]']          
                                                                                              

In [5]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss = {}
metrics = {}
for i in range(6):
    loss[f'output_{i}'] = tf.keras.losses.BinaryFocalCrossentropy()
    metrics[f"output_{i}"] = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [6]:
model.fit(trainSequence,epochs=1,verbose=1)

2023-06-20 20:19:14.832747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-20 20:19:15.121381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 20:19:15.124029: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

 120/1269 [=>............................] - ETA: 7:51 - loss: 0.3616 - output_0_loss: 0.0438 - output_1_loss: 0.1249 - output_2_loss: 0.0088 - output_3_loss: 0.0861 - output_4_loss: 0.0875 - output_5_loss: 0.0105 - output_0_binary_accuracy: 0.9667 - output_1_binary_accuracy: 0.8333 - output_2_binary_accuracy: 1.0000 - output_3_binary_accuracy: 0.9500 - output_4_binary_accuracy: 0.9333 - output_5_binary_accuracy: 1.0000

KeyboardInterrupt: 

In [8]:
preds = model.predict(testSequence,verbose=1)
expected_test = testSequence.get_expected_output()

2023-06-20 20:20:42.378639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


27/27 [==============================] - 3s 104ms/step


In [9]:
expected_test

array([[1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.]], dtype=float32)

In [14]:
score = model.evaluate(testSequence, verbose = 1)

2023-06-20 20:24:47.839692: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


27/27 [==============================] - 3s 114ms/step - loss: 0.1405 - output_0_loss: 9.1828e-04 - output_1_loss: 0.0857 - output_2_loss: 2.9794e-04 - output_3_loss: 0.0441 - output_4_loss: 0.0087 - output_5_loss: 6.8572e-04 - output_0_binary_accuracy: 1.0000 - output_1_binary_accuracy: 0.9259 - output_2_binary_accuracy: 1.0000 - output_3_binary_accuracy: 1.0000 - output_4_binary_accuracy: 1.0000 - output_5_binary_accuracy: 1.0000


In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.14047811925411224
Test accuracy: 0.0009182813810184598


In [17]:
model_name = "ArrangementModel_S_Tier_Only"
model.save(f"Models/{model_name}_{get_timestamp()}.h5")